In [ ]:
# from reso_extract import ms_graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

# Define the path you want to add
new_path = Path("C:\\Users\\andmn\\PycharmProjects\\kverse")

# Convert Path object to string and append it to sys.path
sys.path.append(str(new_path))
from selenium.webdriver.support.wait import WebDriverWait
from kdags.assets.reparation.reso.web_driver import initialize_driver, login_to_reso

# from kdags.assets.reparation.reso_old.document_extractor import (
#     extract_docs_data,
#     click_see_service_order,
# )
# from kdags.assets.reparation.reso_old.file_processor import process_docs_data
from kdags.assets.reparation.reso.web_utils import (
    wait_for_page_load,
    ensure_clickable,
)

# from kdags.assets.reparation.reso.file_utils import save_file_locally
from kdags.assets.reparation.reso.config import DEFAULT_WAIT
import pandas as pd
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException
import time
import os
from datetime import datetime
from kdags.resources.tidyr import DataLake
import polars as pl
import re
import json


from urllib.parse import urlparse  # **** ADDED IMPORT ****
import unicodedata
import requests
from kdags.resources.tidyr import DataLake
from kdags.readr import Readr
from kdags.assets.reparation.reso.service_order_scraper import *
import dagster as dg
from selenium.common.exceptions import (
    NoSuchElementException,
)  # Keep this import for potential future use or other parts of your code

context = dg.build_asset_context()

In [ ]:
driver = initialize_driver()
wait = WebDriverWait(driver, DEFAULT_WAIT)
login_to_reso(driver, wait)
click_presupuesto(driver, wait)

In [ ]:
component_status_df = Readr.Reparation.read_component_status(context)
cc_df = Readr.Planning.read_cc()
filtered_df = (
    cc_df.filter(pl.col("changeout_date").dt.year() >= 2021)
    .join(
        component_status_df.filter((pl.col("site_name").is_in(["MINERA ESCONDIDA"]))).select(
            ["service_order", "customer_work_order", "main_component", "reception_date"]
        ),
        on=["customer_work_order"],
        how="left",
    )
    .sort("changeout_date", descending=True)
    .drop_nulls(subset=["service_order"])
)

In [ ]:
service_orders = filtered_df["service_order"].to_list()
service_orders

In [28]:
QUOTATION_FILE = Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "REPARATION/QUOTATIONS.parquet"
DOCUMENTS_FILE = Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "REPARATION/DOCUMENTS_LIST.parquet"

BATCH_SIZE = 5  # Adjust as needed
QUOTATION_SCHEMA = {
    "service_order": pl.Int64,
    "version": pl.Int64,
    "purchase_order_status": pl.Utf8,
    "date_time_str": pl.Utf8,
    "user": pl.Utf8,
    "amount": pl.Utf8,
    "remarks": pl.Utf8,
    "update_timestamp": pl.Datetime(time_unit="us"),
}
DOCUMENT_SCHEMA = {
    "service_order": pl.Int64,
    "file_name": pl.Utf8,
    "file_title": pl.Utf8,
    "file_subtitle": pl.Utf8,
    "update_timestamp": pl.Datetime(time_unit="us"),
}


# --- Batch Processing Function (process_and_save_batch - remains the same) ---
def process_and_save_batch(
    batch_quotations,
    batch_documents,
    main_quotations_df,
    main_documents_df,
    update_mode,
    quotation_filepath,
    document_filepath,
    processed_sos_in_batch,
):
    """Updates main DFs with batch data ensuring schema order and saves to Parquet."""
    if not batch_quotations and not batch_documents:
        return main_quotations_df, main_documents_df

    quotation_schema_with_ts = QUOTATION_SCHEMA.copy()
    document_schema_with_ts = DOCUMENTS_LIST_SCHEMA.copy()
    quotation_cols_order = list(quotation_schema_with_ts.keys())
    document_cols_order = list(document_schema_with_ts.keys())

    new_quotations_df = (
        pl.DataFrame(batch_quotations, schema_overrides=quotation_schema_with_ts)
        if batch_quotations
        else pl.DataFrame(schema=quotation_schema_with_ts)
    )
    new_documents_df = (
        pl.DataFrame(batch_documents, schema_overrides=document_schema_with_ts)
        if batch_documents
        else pl.DataFrame(schema=document_schema_with_ts)
    )

    if not new_quotations_df.is_empty():
        new_quotations_df = new_quotations_df.select(quotation_cols_order)
    if not new_documents_df.is_empty():
        new_documents_df = new_documents_df.select(document_cols_order)

    if not main_quotations_df.is_empty():
        main_quotations_df = main_quotations_df.select(quotation_cols_order)
    elif not new_quotations_df.is_empty():
        main_quotations_df = pl.DataFrame(schema=quotation_schema_with_ts).select(quotation_cols_order)

    if not main_documents_df.is_empty():
        main_documents_df = main_documents_df.select(document_cols_order)
    elif not new_documents_df.is_empty():
        main_documents_df = pl.DataFrame(schema=document_schema_with_ts).select(document_cols_order)

    if update_mode == "overwrite":
        if not main_quotations_df.is_empty():
            main_quotations_df = main_quotations_df.filter(~pl.col("service_order").is_in(list(processed_sos_in_batch)))
        if not main_documents_df.is_empty():
            main_documents_df = main_documents_df.filter(~pl.col("service_order").is_in(list(processed_sos_in_batch)))

    main_quotations_df = pl.concat([main_quotations_df, new_quotations_df], how="vertical_relaxed")
    main_documents_df = pl.concat([main_documents_df, new_documents_df], how="vertical_relaxed")

    if not main_quotations_df.is_empty():
        main_quotations_df = main_quotations_df.unique(subset=["service_order"], keep="last")
    if not main_documents_df.is_empty():
        main_documents_df = main_documents_df.unique(subset=["service_order", "file_name"], keep="last")

    try:
        quotation_filepath.parent.mkdir(parents=True, exist_ok=True)
        document_filepath.parent.mkdir(parents=True, exist_ok=True)
        main_quotations_df.write_parquet(quotation_filepath)
        main_documents_df.write_parquet(document_filepath)
    except Exception as e:
        print(f"\nCRITICAL: Failed to save batch data to disk! Error: {e}")  # Added newline for visibility

    return main_quotations_df, main_documents_df


# --- Main Processing Function (FIXED with input type conversion) ---


def run_data_extraction(driver, wait, service_orders_input, update_mode="skip"):
    """
    Extracts data, handles specific extraction failures by adding default records,
    saves results in batches with update timestamp.
    Uses UNION logic for skip mode and ensures input service orders are integers.
    """
    if update_mode not in ["skip", "overwrite"]:
        raise ValueError("update_mode must be 'skip' or 'overwrite'")

    # --- FIX: Ensure input service orders are integers ---
    service_orders = []
    try:
        # Attempt to convert all items in the input list to integers
        service_orders = [int(so) for so in service_orders_input]
        print(f"Successfully validated/converted {len(service_orders)} input service orders to integers.")
    except (ValueError, TypeError) as e:
        print(f"CRITICAL ERROR: Could not convert all input service orders to integers: {e}")
        print("Please ensure the 'service_orders_input' list contains only numbers or strings convertible to numbers.")
        sys.exit(1)  # Exit if conversion fails
    # --- END FIX ---

    # Load existing data
    try:
        main_quotations_df = pl.read_parquet(QUOTATION_FILE)
        if "update_timestamp" not in main_quotations_df.columns:
            main_quotations_df = main_quotations_df.with_columns(
                pl.lit(None).cast(pl.Datetime(time_unit="us")).alias("update_timestamp")
            )
        main_quotations_df = main_quotations_df.cast({"update_timestamp": pl.Datetime(time_unit="us")})
        print(f"Loaded existing quotations: {main_quotations_df.shape}")
    except Exception as e:
        print(f"Info: Quotations file not found or error loading ({e}). Starting fresh.")
        main_quotations_df = pl.DataFrame(schema=QUOTATION_SCHEMA)

    try:
        main_documents_df = pl.read_parquet(DOCUMENTS_FILE)
        if "update_timestamp" not in main_documents_df.columns:
            main_documents_df = main_documents_df.with_columns(
                pl.lit(None).cast(pl.Datetime(time_unit="us")).alias("update_timestamp")
            )
        main_documents_df = main_documents_df.cast({"update_timestamp": pl.Datetime(time_unit="us")})
        print(f"Loaded existing documents: {main_documents_df.shape}")
    except Exception as e:
        print(f"Info: Documents file not found or error loading ({e}). Starting fresh.")
        main_documents_df = pl.DataFrame(schema=DOCUMENTS_LIST_SCHEMA)

    # Prepare for 'skip' mode using UNION
    existing_sos = set()
    if update_mode == "skip":
        print("Update mode is 'skip'. Identifying existing service orders...")
        quot_sos = set()
        doc_sos = set()

        if not main_quotations_df.is_empty() and "service_order" in main_quotations_df.columns:
            try:
                quot_sos = set(
                    main_quotations_df["service_order"].drop_nulls().to_list()
                )  # Added drop_nulls() for safety
                print(f"Found {len(quot_sos)} non-null service orders in existing quotations file.")
            except Exception as e:
                print(f"Warning: Could not extract service orders from quotations file: {e}")

        if not main_documents_df.is_empty() and "service_order" in main_documents_df.columns:
            try:
                doc_sos = set(
                    main_documents_df["service_order"].drop_nulls().to_list()
                )  # Added drop_nulls() for safety
                print(
                    f"Found {len(doc_sos)} non-null service orders in existing documents file."
                )  # This should now report correctly
            except Exception as e:
                print(f"Warning: Could not extract service orders from documents file: {e}")

        existing_sos = quot_sos.union(doc_sos)
        print(f"--> Total unique existing service orders identified for skipping: {len(existing_sos)}")
        # --- Optional Debug Print ---
        # if existing_sos:
        #     sample_so_type = type(next(iter(existing_sos)))
        #     print(f"DEBUG: Sample type in existing_sos set: {sample_so_type}")
        # else:
        #     print("DEBUG: existing_sos set is empty.")
        # --- End Optional Debug Print ---

    batch_quotations = []
    batch_documents = []
    processed_sos_in_batch = set()
    total_orders = len(service_orders)  # Use the validated integer list
    print("--- Starting Data Extraction ---")
    skipped_count = 0

    # Use the validated integer list 'service_orders' here
    for i, service_order in enumerate(service_orders):
        print(
            f"Processing SO {i+1}/{total_orders}: {service_order} (Skipped: {skipped_count})",
            end="\r",
        )

        # --- Optional Debug Print ---
        # print(f"\nDEBUG: Checking SO: {service_order} | Type: {type(service_order)}")
        # --- End Optional Debug Print ---

        # The actual skip check (now comparing int vs int)
        if update_mode == "skip" and service_order in existing_sos:
            # --- Optional Debug Print ---
            # print(f"\nDEBUG: Match found! Skipping {service_order}.")
            # --- End Optional Debug Print ---
            skipped_count += 1
            continue
        # else:
        # --- Optional Debug Print ---
        # print(f"\nDEBUG: No match in existing_sos. Processing {service_order}.")
        # --- End Optional Debug Print ---

        # --- Attempt to process this Service Order ---
        now = datetime.now()
        quotation_record_added = False
        document_record_added = False

        try:
            search_service_order(driver, wait, service_order)
            click_see_service_order(wait)

            # --- Quotation Extraction Block ---
            try:
                navigate_to_quotation_tab(driver, wait)
                quotation_data = extract_quotation_details(driver, wait, service_order)
                if quotation_data:
                    quotation_data.pop("download_url", None)
                    quotation_data["service_order"] = service_order  # Already int
                    quotation_data["update_timestamp"] = now
                    for key in QUOTATION_SCHEMA.keys():
                        quotation_data.setdefault(key, None)
                    # Basic Type handling (can be enhanced)
                    for key, polars_type in QUOTATION_SCHEMA.items():
                        if key in quotation_data and quotation_data[key] is not None:
                            try:
                                if polars_type == pl.Int64 and not isinstance(quotation_data[key], int):
                                    quotation_data[key] = int(quotation_data[key])
                                elif polars_type == pl.Utf8 and not isinstance(quotation_data[key], str):
                                    quotation_data[key] = str(quotation_data[key])
                            except (ValueError, TypeError):
                                quotation_data[key] = None  # Set to None on conversion error
                    batch_quotations.append(quotation_data)
                    quotation_record_added = True
                # else: No explicit message needed if None is expected for "no data"

            except Exception as e_quot:
                print(f"\nError during quotation extraction for SO {service_order}: {e_quot}. Adding default record.")

            if not quotation_record_added:
                default_quotation_data = {key: None for key in QUOTATION_SCHEMA.keys()}
                default_quotation_data["service_order"] = service_order
                default_quotation_data["update_timestamp"] = now
                batch_quotations.append(default_quotation_data)
                quotation_record_added = True  # Mark default added

            # --- Document Extraction Block ---
            try:
                navigate_to_documents_tab(wait)
                all_docs = extract_document_links(driver, wait)
                if all_docs:
                    for doc_dict in all_docs:
                        doc_dict.pop("url", None)
                        doc_dict["service_order"] = service_order  # Already int
                        doc_dict["update_timestamp"] = now
                        for key in DOCUMENTS_LIST_SCHEMA.keys():
                            doc_dict.setdefault(key, None)
                        # Basic Type handling (can be enhanced)
                        for key, polars_type in DOCUMENTS_LIST_SCHEMA.items():
                            if key in doc_dict and doc_dict[key] is not None:
                                try:
                                    if polars_type == pl.Int64 and not isinstance(doc_dict[key], int):
                                        doc_dict[key] = int(doc_dict[key])
                                    elif polars_type == pl.Utf8 and not isinstance(doc_dict[key], str):
                                        doc_dict[key] = str(doc_dict[key])
                                except (ValueError, TypeError):
                                    doc_dict[key] = None  # Set to None on conversion error
                        batch_documents.append(doc_dict)
                    document_record_added = True
                # else: No explicit message needed if None/[] is expected for "no data"

            except Exception as e_doc:
                print(f"\nError during document extraction for SO {service_order}: {e_doc}. Adding default record.")

            if not document_record_added:
                default_document_data = {key: None for key in DOCUMENTS_LIST_SCHEMA.keys()}
                default_document_data["service_order"] = service_order
                default_document_data["update_timestamp"] = now
                batch_documents.append(default_document_data)
                document_record_added = True  # Mark default added

            processed_sos_in_batch.add(service_order)

        except Exception as outer_e:
            print(f"\nMajor error processing Service Order {service_order}: {outer_e}")
            if not quotation_record_added:  # Add defaults even on outer error if not already added
                default_quotation_data = {key: None for key in QUOTATION_SCHEMA.keys()}
                default_quotation_data["service_order"] = service_order
                default_quotation_data["update_timestamp"] = now
                batch_quotations.append(default_quotation_data)
            if not document_record_added:
                default_document_data = {key: None for key in DOCUMENTS_LIST_SCHEMA.keys()}
                default_document_data["service_order"] = service_order
                default_document_data["update_timestamp"] = now
                batch_documents.append(default_document_data)
            processed_sos_in_batch.add(service_order)  # Add to batch set even if error occurred

        finally:
            try:
                _ = driver.window_handles
                close_service_order_view(wait)
            except Exception as cleanup_e:
                if "outer_e" not in locals() or "WebDriverException" not in str(outer_e):
                    print(f"\nWarning: Failed to close view for SO {service_order} during cleanup: {cleanup_e}")

        # --- Batch Processing Trigger ---
        is_last_item = i == total_orders - 1
        if len(processed_sos_in_batch) >= BATCH_SIZE or (is_last_item and processed_sos_in_batch):
            print(
                f"\nProcessing and saving batch of {len(processed_sos_in_batch)} service orders (ending with SO {service_order})..."
            )  # Added newline
            main_quotations_df, main_documents_df = process_and_save_batch(
                batch_quotations,
                batch_documents,
                main_quotations_df,
                main_documents_df,
                update_mode,
                QUOTATION_FILE,
                DOCUMENTS_FILE,
                processed_sos_in_batch,
            )
            batch_quotations = []
            batch_documents = []
            processed_sos_in_batch = set()

    print(" " * 100, end="\r")  # Clear last progress line
    print("\n--- Data Extraction Complete ---")
    print(f"Final Quotations DF shape: {main_quotations_df.shape}")
    print(f"Final Documents DF shape: {main_documents_df.shape}")
    print(f"Total Service Orders skipped: {skipped_count}")


run_data_extraction(driver, wait, service_orders, update_mode="skip")

Successfully validated/converted 1537 input service orders to integers.
Loaded existing quotations: (1223, 8)
Loaded existing documents: (5954, 5)
Update mode is 'skip'. Identifying existing service orders...
Found 1223 non-null service orders in existing quotations file.
Found 1223 non-null service orders in existing documents file.
--> Total unique existing service orders identified for skipping: 1223
--- Starting Data Extraction ---
Processing SO 1232/1537: 14122533 (Skipped: 1231)
Error during quotation extraction for SO 14122533: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7FDA71F55+78133]
	GetHandleVerifier [0x00007FF7FDA71FB0+78224]
	(No symbol) [0x00007FF7FD8391BA]
	(No symbol) [0x00007FF7FD88F19D]
	(No symbol) [0x00007FF7FD88F44C]
	(No symbol) [0x00007FF7FD8E23D7]
	(No symbol) [0x00007FF7FD8B719F]
	(No symbol) [0x00007FF7FD8DF21F]
	(No symbol) [0x00007FF7FD8B6F33]
	(No symbol) [0x00007FF7FD880358]
	(No symbol) [0x00007FF7FD8810C3]
	GetHandleVerifier [0x00007FF7FDD3BA8D+

In [ ]:
def click_presupuesto(driver, wait):
    # ---  Clicking the SPAN containing "Presupuesto" ---
    # Use XPath to find the specific <span> element containing the text
    presupuesto_locator = (By.XPATH, "//span[normalize-space(text())='Presupuesto']")
    # Wait for the span element to be present on the page
    presupuesto_span = wait.until(EC.presence_of_element_located(presupuesto_locator))
    # Scroll the element into view to ensure it's not off-screen
    driver.execute_script("arguments[0].scrollIntoView(true);", presupuesto_span)
    time.sleep(0.5)  # Brief pause after scroll
    wait.until(EC.element_to_be_clickable(presupuesto_locator))
    presupuesto_span.click()  # Click the span
    print("Waiting for loading overlay to disappear...")
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".overlay")))
    print("Loading overlay disappeared.")


def search_service_order(driver, wait, service_order):
    """
    Searches for a specific service order on the page and waits for results.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
        service_order: The service order number (int or str) to search for.
    """
    # --- Find the input field, clear it, and enter the service order ---
    # print(f"Locating the service order input field (codigoBuscar)...")
    search_input_locator = (By.ID, "codigoBuscar")
    search_input = wait.until(EC.presence_of_element_located(search_input_locator))
    # print(f"Entering service order: {service_order}")
    search_input.clear()
    search_input.send_keys(str(service_order))  # Ensure input is string
    # ---------------------------------------------------------------------

    # --- Find and click the search button ---
    # print("Locating the search button (buttonBuscar)...")
    search_button_locator = (By.ID, "buttonBuscar")
    search_button = wait.until(EC.element_to_be_clickable(search_button_locator))
    # print("Clicking the search button...")
    search_button.click()
    # -----------------------------------------

    # --- Wait for the filtering results (assuming the overlay reappears) ---
    # print("Waiting for search results filtering overlay to disappear...")
    overlay_locator = (By.CSS_SELECTOR, ".overlay")
    # IMPORTANT: This assumes the *same* overlay with class 'overlay' is used for filtering.
    wait.until(EC.invisibility_of_element_located(overlay_locator))
    # print("Search results filtering complete.")
    # --------------------------------------------------------------------


def click_see_service_order(wait):
    """
    Waits for the results table and clicks the 'See service order' button
    for the first row.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
    """
    # --- Wait for the results table ---
    # print("Waiting for the results table (WebGrid)...")
    webgrid_locator = (By.ID, "WebGrid")
    wait.until(EC.presence_of_element_located(webgrid_locator))
    # print("Results table found.")

    # --- Wait directly for the button to be clickable and click it ---
    # Locator targets the button with class 'verOrdenServicio' in the first row (implicitly)
    see_order_button_locator = (By.CSS_SELECTOR, "button.verOrdenServicio")
    # print("Waiting for the 'See service order' button to be clickable...")
    # This wait implicitly handles presence and visibility before checking clickability
    see_order_button = wait.until(EC.element_to_be_clickable(see_order_button_locator))
    # print("'See service order' button is clickable.")

    # print("Clicking the 'See service order' button...")
    see_order_button.click()  # Using standard click
    # print("Clicked 'See service order' button.")
    # ------------------------------------------------------------------


def navigate_to_quotation_tab(driver, wait):
    """
    Waits for and clicks the 'Quotation' tab, then waits for its content to load.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
    """
    # --- Wait for the Quotation tab button to be ready and click it ---
    # print("Waiting for the 'Quotation' tab button (v-pills-presupuesto-tab) to be clickable...")
    quotation_tab_locator = (By.ID, "v-pills-presupuesto-tab")

    # Wait for the tab button to be clickable
    quotation_tab_button = wait.until(EC.element_to_be_clickable(quotation_tab_locator))
    # print("'Quotation' tab button is clickable.")

    # print("Clicking the 'Quotation' tab button...")
    quotation_tab_button.click()
    # print("Clicked 'Quotation' tab button.")
    # ---------------------------------------------------------------------

    # --- Wait for content within the Quotation tab to load (if necessary) ---
    # Assumes the overlay appears again. Adjust if needed.
    # print("Waiting for Quotation tab content to load (overlay to disappear)...")
    overlay_locator = (By.CSS_SELECTOR, ".overlay")
    wait.until(EC.invisibility_of_element_located(overlay_locator))
    # print("Quotation tab content loaded (overlay disappeared).")
    # --------------------------------------------------------------------------


def extract_quotation_details(driver, wait, service_order):
    """
    Extracts details from the most recent quotation entry displayed. Handles missing Amount field.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
        service_order: The service order number associated with this quotation.

    Returns:
        A dictionary containing the extracted quotation details,
        or None if the main container is not found (though the wait should handle this).
    """

    # --- Locate the main container for quotations ---
    # print("Waiting for Quotation container (divPres1)...")
    quotation_container_locator = (By.ID, "divPres1")
    wait.until(EC.visibility_of_element_located(quotation_container_locator))
    # print("Quotation container found and visible.")

    # --- Locate the first/most recent entry block within the container ---
    # This assumes the most recent entry is always the first child.
    # print("Locating the first quotation entry block...")
    first_entry_locator = (By.CSS_SELECTOR, "#divPres1 > div:first-child > div")
    first_entry_element = wait.until(EC.visibility_of_element_located(first_entry_locator))
    # print("First quotation entry block found and visible.")

    # --- Extract data from the first entry block ---
    quotation_data = {}

    # Version and Purchase Order Status
    # Using try-except here is reasonable as text format variations could cause issues
    try:
        version_po_element = first_entry_element.find_element(By.CSS_SELECTOR, "div > span > i")
        version_po_text = version_po_element.text
        # print(f"Found Version/PO text block: {version_po_text}")
        version_match = re.search(r"Version (\d+):", version_po_text)
        quotation_data["version"] = int(version_match.group(1)) if version_match else None
        po_status_match = re.search(r"Purchase Order:\s*([^\n]+)", version_po_text, re.IGNORECASE)
        quotation_data["purchase_order_status"] = po_status_match.group(1).strip() if po_status_match else None
    except Exception as e:  # Catching broader exception for text processing issues
        print(f"Warning: Could not extract Version/PO details: {e}")
        quotation_data["version"] = None
        quotation_data["purchase_order_status"] = None

    # print(f"Extracted Version: {quotation_data['version']}")
    # print(f"Extracted PO Status: {quotation_data['purchase_order_status']}")

    # Locate the block containing the most recent details
    # The HTML shows multiple 'detail' blocks, the first one seems to be the latest approval/action
    details_container = first_entry_element.find_element(By.CSS_SELECTOR, "div > div")
    # This selects the last div within the first div inside details_container
    # Based on HTML: selects the div containing date/user/remarks for the first entry (Karen Guerra)
    most_recent_detail_block = details_container.find_element(By.CSS_SELECTOR, "div:first-child > div:last-child")

    # Date/Time
    try:
        date_block = most_recent_detail_block.find_element(By.CSS_SELECTOR, "i.fecha")
        day = date_block.find_element(By.TAG_NAME, "dia").text
        month = date_block.find_element(By.TAG_NAME, "mes").text
        year_time = date_block.find_element(By.TAG_NAME, "año").text
        quotation_data["date_time_str"] = f"{day} {month} {year_time}".strip()
    except NoSuchElementException:
        print("Warning: Date/Time element not found.")
        quotation_data["date_time_str"] = None

    # User
    try:
        # Using normalize-space() helps handle variations in whitespace
        user_element = most_recent_detail_block.find_element(By.XPATH, ".//span[contains(normalize-space(), 'User:')]")
        # Split carefully, handling potential lack of space after colon
        user_text_parts = user_element.text.split("User:", 1)
        quotation_data["user"] = user_text_parts[1].strip() if len(user_text_parts) > 1 else None
    except NoSuchElementException:
        print("Warning: User element not found.")
        quotation_data["user"] = None

    # --- MODIFIED AMOUNT EXTRACTION ---
    # print("Locating Amount element (if present)...")
    # Use find_elements (plural) to get a list
    amount_elements = most_recent_detail_block.find_elements(
        By.XPATH, ".//span[contains(normalize-space(), 'Amount:')]"
    )

    if amount_elements:
        # If the list is not empty, the element exists
        amount_element = amount_elements[0]  # Get the first matching element
        amount_text = amount_element.text
        # print(f"Found Amount text: {amount_text}")
        # Split carefully
        amount_parts = amount_text.split("Amount:", 1)
        if len(amount_parts) > 1:
            quotation_data["amount"] = amount_parts[1].strip()
        else:
            print("Warning: 'Amount:' text found, but no value after it.")
            quotation_data["amount"] = None  # Or set to "" or handle as needed
    else:
        # If the list is empty, the element was not found
        print("Amount element not found, setting amount to None.")
        quotation_data["amount"] = None
    # --- END OF MODIFIED AMOUNT EXTRACTION ---

    # Remarks
    try:
        remarks_element = most_recent_detail_block.find_element(By.CSS_SELECTOR, "span.overflowText")
        remarks_full_text = remarks_element.text
        # Split carefully, handling case where "Remarks:" might be absent but text exists
        remarks_parts = remarks_full_text.split("Remarks:", 1)
        if len(remarks_parts) > 1:
            quotation_data["remarks"] = remarks_parts[1].strip()
        else:
            # If "Remarks:" isn't present, take the whole text, maybe log a warning
            print("Warning: 'Remarks:' prefix not found in remarks element. Using full text.")
            quotation_data["remarks"] = remarks_full_text.strip()
    except NoSuchElementException:
        print("Warning: Remarks element not found.")
        quotation_data["remarks"] = None

    # Add the service order
    quotation_data["service_order"] = service_order

    # Extract Download URL
    # print("Extracting download URL...")
    try:
        download_link_element = first_entry_element.find_element(By.TAG_NAME, "a")
        download_url = download_link_element.get_attribute("href")
        quotation_data["download_url"] = download_url
        # print(f"Found Download URL: {download_url}")
    except NoSuchElementException:
        print("Warning: Download link element not found.")
        quotation_data["download_url"] = None

    return quotation_data


def navigate_to_documents_tab(wait):
    """
    Waits for and clicks the 'Documents' tab, then waits for its content to load.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
    """
    # --- Wait for the Documents tab button to be ready and click it ---
    # print("Waiting for the 'Documents' tab button (v-pills-documentos-tab) to be clickable...")
    documents_tab_locator = (By.ID, "v-pills-documentos-tab")

    # Wait for the tab button to be clickable
    documents_tab_button = wait.until(EC.element_to_be_clickable(documents_tab_locator))
    # print("'Documents' tab button is clickable.")

    # print("Clicking the 'Documents' tab button...")
    documents_tab_button.click()
    # print("Clicked 'Documents' tab button.")
    # ---------------------------------------------------------------------

    # --- Wait for content within the Documents tab to load (if necessary) ---
    # Assume the overlay appears again. Adjust if needed.
    # print("Waiting for Documents tab content to load (overlay to disappear)...")
    overlay_locator = (By.CSS_SELECTOR, ".overlay")
    wait.until(EC.invisibility_of_element_located(overlay_locator))
    # print("Documents tab content loaded (overlay disappeared).")
    # You might need to wait for a specific element *within* the Documents tab instead, e.g.:
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#v-pills-documentos .some-element")))
    # --------------------------------------------------------------------------


def extract_document_links(driver, wait):

    documents_list = []
    documents_container_locator = (By.ID, "v-pills-documentos")
    document_link_locator = (By.CSS_SELECTOR, "#v-pills-documentos a.documento")

    # print("Waiting for the documents container to be visible...")
    wait.until(EC.visibility_of_element_located(documents_container_locator))
    # print("Documents container is visible.")

    # print("Waiting for document links to be present...")
    wait.until(EC.presence_of_element_located(document_link_locator))
    # print("Document links are present.")

    document_elements = driver.find_elements(*document_link_locator)
    # print(f"Found {len(document_elements)} document links.")

    if not document_elements:
        # print("No documents found in the panel.")
        return documents_list

    # Extract info for each document
    for i, element in enumerate(document_elements):
        # print(f"Processing document {i+1}...")
        doc_info = {}
        url = element.get_attribute("href")
        doc_info["url"] = url

        # **** ADDED: Extract Filename from URL ****
        file_name = None
        if url:
            try:
                parsed_url = urlparse(url)
                url_path = parsed_url.path
                # Use pathlib to reliably get the last part of the path
                file_name = Path(url_path).name
            except Exception as e:
                print(f"  Error parsing URL to get filename for doc {i+1}: {e}")
                file_name = None  # Ensure it's None if parsing failed

        # **** ADDED: Assertion for filename ****
        assert file_name, f"Could not extract filename from URL for doc {i+1}: {url}"
        doc_info["file_name"] = file_name
        # *****************************************

        file_title = None
        file_subtitle = None

        # Find all direct child spans of the link element
        all_spans = element.find_elements(By.XPATH, "./span")  # Direct child spans

        if all_spans:
            # Title Logic (same as before)
            if all_spans[0].text and "File Title:" not in all_spans[0].text.strip():
                file_title = all_spans[0].text.strip()
            elif len(all_spans) > 1 and all_spans[1].text and "File Title:" not in all_spans[1].text.strip():
                file_title = all_spans[1].text.strip()

            # Subtitle Logic (same as before)
            for idx, span in enumerate(all_spans):
                span_text = span.text.strip()
                if "File Title:" in span_text and idx + 1 < len(all_spans):
                    file_subtitle = all_spans[idx + 1].text.strip()
                    if file_title is None and idx > 0:
                        file_title = all_spans[idx - 1].text.strip()
                    break

        # Fallback title (same as before)
        if file_title is None:
            full_text = element.text
            lines = [line.strip() for line in full_text.split("\n") if line.strip() and "(See more" not in line]
            file_title = " ".join(lines) if lines else "Unknown Title"

        # Assign to dictionary
        doc_info["file_title"] = file_title.replace("\n", " ").strip() if file_title else None
        doc_info["file_subtitle"] = file_subtitle.replace("\n", " ").strip() if file_subtitle else None

        # print(f"  Title: {doc_info['file_title']}")
        # print(f"  Subtitle: {doc_info['file_subtitle']}")
        # print(f"  Filename: {doc_info['file_name']}")  # Added print
        # print(f"  URL: {doc_info['url']}")

        # Only add if we got a URL (filename assertion already checked)
        documents_list.append(doc_info)

    # print(f"Extracted {len(documents_list)} document records.")
    return documents_list


def close_service_order_view(wait):
    """
    Clicks the 'closePCI' element to close the detailed service order view
    and waits for the search input field on the previous screen to appear.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The WebDriverWait instance.
    """
    # --- Wait for the close button/div to be clickable ---
    # print("Waiting for the close button (closePCI) to be clickable...")
    close_button_locator = (By.ID, "closePCI")

    # Wait for the element to be clickable
    close_button = wait.until(EC.element_to_be_clickable(close_button_locator))
    # print("Close button (closePCI) is clickable.")

    # print("Clicking the close button...")
    close_button.click()
    # print("Clicked close button.")
    # ----------------------------------------------------

    # --- Wait for navigation back to the previous screen ---
    # Assuming returning to the search screen means the 'codigoBuscar' input is present again
    # print("Waiting to return to the search view (waiting for 'codigoBuscar')...")
    previous_screen_element_locator = (By.ID, "codigoBuscar")
    wait.until(EC.presence_of_element_located(previous_screen_element_locator))
    # print("Returned to the search view (found 'codigoBuscar').")
    # You might need to add a wait for an overlay to disappear here as well, if one appears:
    # print("Waiting for overlay to disappear after closing...")
    # wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".overlay")))
    # print("Overlay disappeared.")
    # -----------------------------------------------------

In [26]:
print(pd.read_parquet(QUOTATION_FILE).query("service_order==14153742").to_dict())
print(pd.read_parquet(DOCUMENTS_FILE).query("service_order==14153742").to_dict())

{'service_order': {800: 14153742}, 'version': {800: nan}, 'purchase_order_status': {800: None}, 'date_time_str': {800: None}, 'user': {800: None}, 'amount': {800: None}, 'remarks': {800: None}, 'update_timestamp': {800: Timestamp('2025-04-08 13:38:40.764789')}}
{'service_order': {2500: 14153742}, 'file_name': {2500: '14153742_314981_INI_APRO.pdf'}, 'file_title': {2500: 'Informe Tecnico Preliminar [Revisado]'}, 'file_subtitle': {2500: None}, 'update_timestamp': {2500: Timestamp('2025-04-08 13:38:40.764789')}}


In [29]:
pd.read_parquet(QUOTATION_FILE)

,service_order,version,purchase_order_status,date_time_str,user,amount,remarks,update_timestamp
0,13012740,1.0,NO APLICA,9 December 2024 16:21:03,Ingrid Meza Caniuqueo,None,"Se aprueba PPTO. costo $0, Garantía Taller con...",2025-04-08 14:24:14.679094
1,14120240,1.0,4507093760,24 August 2021 08:40:42,RICARDO VEGA FLORES,None,"Se aprueba presupuesto de reparación, valor ne...",2025-04-08 16:23:11.205651
2,14125584,NaN,None,None,None,None,None,2025-04-08 15:31:56.341032
3,14129803,NaN,None,None,None,None,None,2025-04-08 15:28:45.275681
4,14118120,2.0,4507090931,6 April 2021 23:16:11,KAREN GUERRA GUARDA,None,Se aprueba ppto por $11.992.672 CLP.- COMPONEN...,2025-04-08 16:32:59.136065
...,...,...,...,...,...,...,...,...
1505,14136820,1.0,PENDIENTE,8 July 2023 16:09:02,GUILLERMO PEREDA RAMOS,"$ 46,578.67 USD","Se aprueba presupuesto USD 46.578,67, overhaul.",2025-04-08 14:48:31.409367
1506,14136558,1.0,PENDIENTE,17 July 2023 09:33:08,GUILLERMO PEREDA RAMOS,"$ 48,198.25 USD","Se aprueba presupuesto USD 48.198,25 , overhaul.",2025-04-08 14:50:03.119785
1507,14135784,1.0,PENDIENTE,22 June 2023 13:04:19,GUILLERMO PEREDA RAMOS,"$ 35,646.73 USD","Se aprueba presupuesto USD 35.646,73, overhaul.",2025-04-08 14:52:40.685670
1508,14125024,NaN,None,None,None,None,None,2025-04-08 15:32:11.493747


In [ ]:
service_order = 14153742
# --- Perform Web Actions and Extraction ---
search_service_order(driver, wait, service_order)
click_see_service_order(wait)
navigate_to_quotation_tab(driver, wait)
quotation_data = extract_quotation_details(driver, wait, service_order)
navigate_to_documents_tab(wait)
all_docs = extract_document_links(driver, wait)
close_service_order_view(wait)

# *** Get current timestamp ***
now = datetime.now()

# --- Data Preparation (Add timestamp) ---
if quotation_data:
    quotation_data.pop("download_url", None)
    quotation_data["service_order"] = service_order
    quotation_data["update_timestamp"] = now  # Add timestamp
    # Ensure all keys exist, including the new one
    for key in QUOTATION_SCHEMA.keys():
        quotation_data.setdefault(key, None)
    # batch_quotations.append(quotation_data)

if all_docs:
    for doc_dict in all_docs:
        doc_dict.pop("url", None)
        doc_dict["service_order"] = service_order
        doc_dict["update_timestamp"] = now  # Add timestamp
        # Ensure all keys exist, including the new one
        for key in DOCUMENT_SCHEMA.keys():
            doc_dict.setdefault(key, None)
        # batch_documents.append(doc_dict)

# processed_sos_in_batch.add(service_order)

In [ ]:
quotations_details = []
docs = []

# **** ADDED: Use enumerate for index and value ****
for i, service_order in enumerate(service_orders):
    # **** MODIFIED: Print progress ****
    print(f"\n--- Processing service order {i+1} of {service_orders.__len__()}: {service_order} ---")

    # --- Steps for each service order ---
    # print(f"  Step 1: Searching for SO...")
    search_service_order(driver, wait, service_order)

    # print(f"  Step 2: Clicking 'See Service Order'...")
    # Pass driver explicitly if needed by the function signature
    click_see_service_order(wait)

    # print(f"  Step 3: Navigating to Quotation Tab...")
    # Pass driver explicitly if needed by the function signature
    navigate_to_quotation_tab(driver, wait)
    # print(f"  Step 4: Extracting Quotation Details...")
    quotation_data = extract_quotation_details(driver, wait, service_order)
    quotations_details.append(quotation_data)  # Append details
    # print(f"  Step 5: Navigating to Documents Tab...")
    # Pass driver explicitly if needed by the function signature
    navigate_to_documents_tab(wait)
    # print(f"  Step 6: Extracting Document Links...")
    all_docs = extract_document_links(driver, wait)
    docs.append(all_docs)
    close_service_order_view(wait)